## Installation

In [1]:
!pip install -q gradio huggingface_hub

     |████████████████████████████████| 6.1 MB 29.0 MB/s 
     |████████████████████████████████| 120 kB 63.7 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 270 kB 67.1 MB/s 
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 112 kB 72.9 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 212 kB 57.2 MB/s 
     |████████████████████████████████| 2.3 MB 50.7 MB/s 
     |████████████████████████████████| 57 kB 5.3 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 9.8 MB/s 
     |████████████████████████████████| 68 kB 6.5 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 856 kB 65.3 MB/s 
     |████████████████████████████████| 594 kB 51.6 MB/s 
     |██████████████████████████

## Fetch a model from the HF Hub

In [6]:
from huggingface_hub import from_pretrained_keras


model_ckpt = "chansung/test-unet"

MODEL = from_pretrained_keras(model_ckpt)

config.json not found in HuggingFace Hub


## Fetch a test image

In [7]:
!wget https://i.ibb.co/whmGJr4/test-image.jpg -O test-image.jpg

--2022-09-13 02:25:21--  https://i.ibb.co/whmGJr4/test-image.jpg
Resolving i.ibb.co (i.ibb.co)... 51.210.32.106, 217.182.228.53, 51.210.32.103, ...
Connecting to i.ibb.co (i.ibb.co)|51.210.32.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182420 (178K) [image/jpeg]
Saving to: ‘test-image.jpg’

test-image.jpg      100%[===================>] 178.14K   505KB/s    in 0.4s    

2022-09-13 02:25:22 (505 KB/s) - ‘test-image.jpg’ saved [182420/182420]



## Utilities

In [8]:
from PIL import Image 
import numpy as np 
import tensorflow as tf 


RESOLTUION = 128
# MODEL = tf.keras.models.load_model(model_path)


def preprocess_input(image: Image) -> tf.Tensor:
    image = np.array(image)
    image = tf.convert_to_tensor(image)

    image = tf.image.resize(image, (RESOLTUION, RESOLTUION))
    image = image / 255

    return tf.expand_dims(image, 0)


# The below utilities (sidewalk_palette(), get_seg_overlay()) are from:
# https://github.com/deep-diver/semantic-segmentation-ml-pipeline/blob/main/notebooks/inference_from_SavedModel.ipynb

def sidewalk_palette():
    """Sidewalk palette that maps each class to RGB values."""
    return [
        [0, 0, 0],
        [216, 82, 24],
        [255, 255, 0],
        [125, 46, 141],
        [118, 171, 47],
        [161, 19, 46],
        [255, 0, 0],
        [0, 128, 128],
        [190, 190, 0],
        [0, 255, 0],
        [0, 0, 255],
        [170, 0, 255],
        [84, 84, 0],
        [84, 170, 0],
        [84, 255, 0],
        [170, 84, 0],
        [170, 170, 0],
        [170, 255, 0],
        [255, 84, 0],
        [255, 170, 0],
        [255, 255, 0],
        [33, 138, 200],
        [0, 170, 127],
        [0, 255, 127],
        [84, 0, 127],
        [84, 84, 127],
        [84, 170, 127],
        [84, 255, 127],
        [170, 0, 127],
        [170, 84, 127],
        [170, 170, 127],
        [170, 255, 127],
        [255, 0, 127],
        [255, 84, 127],
        [255, 170, 127],
    ]


def get_seg_overlay(image, seg):
    color_seg = np.zeros(
        (seg.shape[0], seg.shape[1], 3), dtype=np.uint8
    )  # height, width, 3
    palette = np.array(sidewalk_palette())
    
    for label, color in enumerate(palette):
        color_seg[seg == label, :] = color

    # Show image + mask
    img = np.array(image) * 0.5 + color_seg * 0.5

    img *= 255
    img = np.clip(img, 0, 255)
    img = img.astype(np.uint8)
    return img


def run_model(image: Image) -> tf.Tensor:
    preprocessed_image = preprocess_input(image)
    prediction = MODEL.predict(preprocessed_image)

    seg_mask = tf.math.argmax(prediction, -1)
    seg_mask = tf.squeeze(seg_mask)
    return seg_mask


def get_predictions(image: Image):
    predicted_segmentation_mask = run_model(image)
    preprocessed_image = preprocess_input(image)
    preprocessed_image = tf.squeeze(preprocessed_image, 0)
    
    pred_img = get_seg_overlay(preprocessed_image.numpy(), predicted_segmentation_mask.numpy())
    return Image.fromarray(pred_img)

## Gradio demo

In [ ]:
import gradio as gr

title = "Simple demo for a semantic segmentation model trained on the Sidewalks dataset."

description = """

Note that the outputs obtained in this demo won't be state-of-the-art. The underlying project has a different objective focusing more on the ops side of
deploying a semantic segmentation model. For more details, check out the repository: https://github.com/deep-diver/semantic-segmentation-ml-pipeline/.

"""

demo = gr.Interface(
    get_predictions,
    gr.inputs.Image(type="pil"),
    "pil",
    allow_flagging="never",
    title=title,
    description=description,
    examples=[["test-image.jpg"]]
)

demo.launch(debug=True)

To add more examples, do - `examples=[["test-image.jpg"], ["image2.jpg"], ["sample.png"], ...]`.